In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('../input/pokemon-challenge/pokemon.csv')

In [ ]:
data.info() #Veri setinin içeriği hakkında biraz ön bilgi kazanalım.

Buradan yola çıkarak veri setinde 12 tane kolon olduğunu bunlardan 8 tanesinin integer, 3 tanesinin object ve 1 tanesinin de boolean değişken olduğunu söyleyebiliriz.

In [ ]:
data.count() # Veri setinde kaç adet gözlem ve değişken olduğunu yazdıralım.

Name ve Type 2 isimli kolonlarda nan değerler olduğunu gözlemleyebiliriz. Bu kolonlarla çalışma yapmamız gerekirse nan değerler bize sorun yaratabilir, bu yüzden bu değerleri düzeltmemiz gerekebilir.

In [ ]:
data.isna().sum() #Eksik gözlemlerin kaç tane olduğuna bakalım.

Name değişkeninde 1, Type 2 değişkeninde 386 nan değer olduğu gözükür. Diğer değişkenlerde nan değer yoktur.

In [ ]:
data.head() #Veri setinin ilk 5 gözlemini görüntüleyelim.

In [ ]:
data.tail() #Veri setinin son 5 gözlemini görüntüleyelim.

In [ ]:
data.sample(10) #Veri setinden restgele 10 gözlem görüntüleyelim.

Kod satırını her run edişimizde farklı sonuçlar gözlemleriz.

In [ ]:
data["Type 1"].sort_values().unique() #Type 1 kolonunun benzersiz değerlerini görüntüleyelim ve sıralayalım.


In [ ]:
data["Type 2"].sort_values().unique() #Type 2 kolonunun benzersiz değerlerini görüntüleyelim ve sıralayalım.

 Type 1 ve Type 2 kolonlarını incelediğimizde bu iki kolonun içerdiği benzersiz değerlerin aynı olduğunu gözlemleyebilriz. Type 2 kolonunda girilmemiş değerler olduğundan Type 1 kolonundan farklı olarak nan değeri de bulunur.

In [ ]:
data.describe().T #Sadece sayısal verileri açıklar.

Veri setinin istatiklerine biraz göz gezdirirsek standart sapmalarının yüksek olduğunu ve minimum, maksimum açıklığının yüksek olduğunu söyleyebiliriz.

Üzerlerinde daha rahat çalışabilmek için kolon isimlerini düzenleyelim.

In [ ]:
data = data.rename(columns = {"Type 1":"Type1","Type 2":"Type2","Sp. Atk":"SpAtk","Sp. Def":"SpDef"})
data

Veri setimizin korelasyonunu inceleyelim.

In [ ]:
corr = data.corr()
corr

In [ ]:
f,ax=plt.subplots(figsize=(18,18))
sns.heatmap(corr, annot=True, ax=ax)
plt.show()

Isı haritasını incelediğimizde SpDef değeri yüksek olan pokemonların Defense ve SpAtk değerlerinin de yüksek olduğunu görürüz. Yani SpDef değeriyle Defense değeri arasında güçlü ve pozitif bir ilişki vardır. Aynı zamanda SpDef değeriyle SpAtk değeri arasında da güçlü ve pozitif bir ilişki bulunmaktadır.

Grafikler yardımıyla Defense-SpDef ve SpDef-SpAtk değerleri arasındaki ilişkiyi inceleyelim.

In [ ]:
data.plot(kind="scatter", x="Defense", y="SpDef", color="green")
plt.xlabel("Defense")
plt.xlabel("SpDef")
plt.title("Defense - SpDef Scatter Plot")
plt.show()

In [ ]:
data.plot(kind="scatter", x="SpAtk", y="SpDef",color="red")
plt.xlabel("SpAtk")
plt.xlabel("SpDef")
plt.title("SpAtk - SpDef Scatter Plot")
plt.show()

In [ ]:
data.groupby(["Type1"]).describe()["Attack"].sort_values('mean',ascending=False) #Pokemonların tiplerine göre ortalama attack değerlerini sıralayalım.

In [ ]:
data.groupby(["Type1"]).describe()["Defense"].sort_values('mean',ascending=False)

In [ ]:
data.groupby(["Type1"]).describe()["HP"].sort_values('mean',ascending=False)

In [ ]:
data[(data["Type1"] == "Dragon")] #Dragon tipindeki pokemonları gözlemleyelim.

In [ ]:
data[(data["Type1"] == "Fairy")] #Fairy tipindeki pokemonları gözlemleyelim.

In [ ]:
#Veri setiyle daha kolay uğraşabilmek için biraz küçültelim ve 6 tip pokemon üzerinden ilerleyelim.
dragon = data[(data["Type1"] == "Dragon")]
rock = data[(data["Type1"] == "Rock")]
fire = data[(data["Type1"] == "Fire")]
water = data[(data["Type1"] == "Water")]
grass = data[(data["Type1"] == "Grass")]            
fairy = data[(data["Type1"] == "Fairy")]

Veri setini seaborn ve matplot kütüphanelerinden yardım alarak görselleştirelim.

In [ ]:
smallData = pd.concat([dragon,rock,fire,water,grass,fairy])
sns.violinplot(x = "Type1", y = "Attack", data = smallData);

In [ ]:
plt.scatter(dragon.Name, dragon.Attack, color="#ff6600", label="Dragon" )
plt.scatter(rock.Name, rock.Attack, color="#993300", label="Rock" )
plt.scatter(fire.Name, fire.Attack, color="#ff1a1a", label="Fire" )
plt.scatter(water.Name, water.Attack, color="#0066ff", label="Water" )
plt.scatter(grass.Name, grass.Attack, color="#33cc33", label="Grass" )
plt.scatter(fairy.Name, fairy.Attack, color="#ff0066", label="Fairy")



Görselleştirdiğimiz verileri incelersek dragon tipi pokemonların attack değerindeki yüksekliği daha iyi anlayabiliriz.

Veri setindeki en güçlü pokemon türünü bulmaya çalışalım. Bulma yollarından biri pokemon tür niceliklerinin ortalamanın ne kadar altında veya üstünde olduğuna bakmak olabilir.

In [ ]:
dicts={}
smallData2 = smallData.loc[:,"HP":"Speed"]
liste=list()
for each in smallData2.columns:
    smallData.groupby(["Type1"])[each].mean()
    for i in range(6):
        liste.append(smallData.groupby(["Type1"])[each].describe().iloc[i,1]-smallData2.describe().iloc[1,i])
    dicts[each] = liste.copy()
    liste.clear()
    

Oluşturacağımız yeni veri setinde pokemon özelliklerinin ortalamadan farklarını tutalım.

In [ ]:
typeMean=pd.DataFrame(dicts)
typeMean.index=smallData.Type1.sort_values().unique().copy()
typeMean

In [ ]:
typeMean = typeMean.T.describe()
typeMean

In [ ]:
plt.plot(typeMean.columns, typeMean.iloc[1,:], color="#ff6600", label="Dragon" )


Bu verilere ve grafiğe bakıldığında; dragon tipi pokemonların incelediğimiz diğer pokemonlara karşı daha güçlü olduğunu söyleyebiliriz.